<a href="https://www.kaggle.com/code/ibrahimnibrahim/sports-ball-classifier?scriptVersionId=244285171" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    print(dirname)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input
/kaggle/input/sports-balls-multiclass-image-classification
/kaggle/input/sports-balls-multiclass-image-classification/test
/kaggle/input/sports-balls-multiclass-image-classification/test/rugby_ball
/kaggle/input/sports-balls-multiclass-image-classification/test/hockey_ball
/kaggle/input/sports-balls-multiclass-image-classification/test/volleyball
/kaggle/input/sports-balls-multiclass-image-classification/test/golf_ball
/kaggle/input/sports-balls-multiclass-image-classification/test/football
/kaggle/input/sports-balls-multiclass-image-classification/test/shuttlecock
/kaggle/input/sports-balls-multiclass-image-classification/test/baseball
/kaggle/input/sports-balls-multiclass-image-classification/test/american_football
/kaggle/input/sports-balls-multiclass-image-classification/test/hockey_puck
/kaggle/input/sports-balls-multiclass-image-classification/test/table_tennis_ball
/kaggle/input/sports-balls-multiclass-image-classification/test/billiard_ball
/kaggle/input/sports-ba

In [2]:
import tensorflow as tf

batch_size = 64
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
    '/kaggle/input/sports-balls-multiclass-image-classification/train',
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    '/kaggle/input/sports-balls-multiclass-image-classification/test',
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

Found 7328 files belonging to 15 classes.
Found 1841 files belonging to 15 classes.


In [3]:
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

Number of devices: 2


In [4]:
train_ds = train_ds.cache().shuffle(2000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

train_ds = strategy.experimental_distribute_dataset(train_ds)
test_ds = strategy.experimental_distribute_dataset(test_ds)

In [5]:
from tensorflow.keras.applications import EfficientNetB3

with strategy.scope():
    base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
    base_model.trainable = False

    
    inputs = tf.keras.Input(shape=(img_height, img_width, 3))
    x = base_model(inputs, training=False)  # Pass inputs through the base model
    x = tf.keras.layers.Flatten()(x)  
    x = tf.keras.layers.Dense(512, activation='relu')(x)  # Fully connected layer
    x = tf.keras.layers.Dense(512, activation='relu')(x)  # Fully connected layer
    outputs = tf.keras.layers.Dense(15, activation='softmax')(x)  # Output layer
    
    # Create the final model
    model = tf.keras.Model(inputs, outputs)
    
    model.compile(loss='sparse_categorical_crossentropy',
                    optimizer=tf.keras.optimizers.Adam(),
                    metrics=['accuracy'])

model.summary()

43941136/43941136 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ efficientnetb3 (Functional)          │ (None, 7, 7, 1536)          │      10,783,535 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 75264)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │      38,535,680 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 512)                 │         262,656 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 15)                  │           7,695 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 49,589,566 (189.17 MB)

 Trainable params: 38,806,031 (148.03 MB)

 Non-trainable params: 10,783,535 (41.14 MB)

In [6]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',  # Monitor validation loss
    patience=5,          # Stop after 5 epochs without improvement
    min_delta=0.001,     # Minimum change to qualify as improvement
    restore_best_weights=True  # Restore the best model weights
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy',  # Monitor validation loss
    factor=0.5,         # Reduce learning rate by a factor of 0.1
    patience=3,         # Wait 3 epochs before reducing the learning rate
    min_lr=1e-6,        # Minimum learning rate
    mode='max',         # Minimize the monitored metric
    verbose=1           # Print a message when the learning rate is reduced
)

In [7]:
model.fit(train_ds, epochs=30,validation_data=test_ds,callbacks=[early_stopping,reduce_lr])

Epoch 1/30
115/115 ━━━━━━━━━━━━━━━━━━━━ 58s 319ms/step - accuracy: 0.5647 - loss: 3.5818 - val_accuracy: 0.8339 - val_loss: 0.6269 - learning_rate: 0.0010
Epoch 2/30
115/115 ━━━━━━━━━━━━━━━━━━━━ 27s 229ms/step - accuracy: 0.9111 - loss: 0.3067 - val_accuracy: 0.8545 - val_loss: 0.5189 - learning_rate: 0.0010
Epoch 3/30
115/115 ━━━━━━━━━━━━━━━━━━━━ 27s 235ms/step - accuracy: 0.9503 - loss: 0.1735 - val_accuracy: 0.8578 - val_loss: 0.6522 - learning_rate: 0.0010
Epoch 4/30
115/115 ━━━━━━━━━━━━━━━━━━━━ 27s 237ms/step - accuracy: 0.9638 - loss: 0.1934 - val_accuracy: 0.8806 - val_loss: 0.6208 - learning_rate: 0.0010
Epoch 5/30
115/115 ━━━━━━━━━━━━━━━━━━━━ 27s 234ms/step - accuracy: 0.9625 - loss: 0.1205 - val_accuracy: 0.8480 - val_loss: 0.7923 - learning_rate: 0.0010
Epoch 6/30
115/115 ━━━━━━━━━━━━━━━━━━━━ 27s 232ms/step - accuracy: 0.9586 - loss: 0.1473 - val_accuracy: 0.8588 - val_loss: 0.7157 - learning_rate: 0.0010
Epoch 7/30
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step - accuracy: 0.95

In [8]:
model.evaluate(test_ds)

29/29 ━━━━━━━━━━━━━━━━━━━━ 4s 137ms/step - accuracy: 0.8773 - loss: 0.8040


[0.7295082211494446, 0.895765483379364]